In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install folium

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from tabulate import tabulate
from geopy.geocoders import Nominatim
import folium
import re  ##match patterns string
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('We are ready!')

We are ready!


In [3]:
url    = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webdoc =requests.get(url)
canada = BeautifulSoup(webdoc.content,'lxml')
##table
table=canada.find_all('table')
df=pd.read_html(str(table))[0]
df=pd.DataFrame(df)
df.columns=['PostalCode','Borough','Neighborhood']
df.drop([0], inplace=True)
# data
df=df[df.Borough != 'Not assigned']
index = np.asarray(list(range(0,len(df['PostalCode']))))
df=df.set_index(index)

df_toronto=pd.DataFrame()
df_toronto=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

##Cleaning and putting in one dataframe
df_toronto=df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_toronto.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [0]:
na_neigh_rows = df_toronto.Neighborhood == 'Not assigned'
df_toronto.loc[na_neigh_rows, 'Neighborhood'] = df_toronto.loc[na_neigh_rows, 'Borough']
# df_toronto #[na_neigh_rows]

In [5]:
df_toronto_final = df_toronto
df_toronto_final.shape

(103, 3)

In [7]:
##Getting csv
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Awesome, you got it!')
coor= pd.read_csv('toronto_coordinates.csv')
#coor.shape

Awesome, you got it!


In [8]:
##Dataframe
df_coor=pd.DataFrame(coor)
df_coor.columns=['PostalCode','Borough','Neighborhood']
##Merging dataframes
df_final=pd.merge(df_toronto_final,df_coor,on='PostalCode')
df_final.columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude']
df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
